# ML_PROFILING NOTEBOOK
## It's for doing the ml profiling once the data is analized in DATA_PROFILING NOTEBOOK

In [ ]:

%load_ext autoreload
%autoreload 2
# The %load_ext autoreload and %autoreload 2 magic commands are used to automatically 
# reload modules when they are changed. This can be useful when you are developing code 
# in an interactive environment, as it allows you to see the changes you make to your modules 
# without having to restart the kernel.


    

In [ ]:

# Load the necesary libraries and print the acces to conf file config.yaml

from omegaconf import OmegaConf
import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
import os


# for global initialization: NOT RECOMMENDED
#initialize(version_base=None, config_path="../src/conf")
#compose(config_name='config')

print("\n\nthis shows the cfg file:")
with initialize(version_base=None, config_path="../src/conf"):
    cfg = compose(config_name='config')
    print(cfg)

import pandas as pd
import numpy as np


In [ ]:
#loading the dataset
data = pd.read_csv(os.path.join('../data/raw', cfg.file_names.raw_file), 
                  # encoding=cfg.general_ml.encoding,
                  )

In [ ]:
data.columns

In [ ]:
#cutting dataset to this experiment
data=data[[
    'estado',
    'municipio',
    'businesses',
    'employees',
    'income',
    'Payroll'
]]
data

In [ ]:
#lets find ceros
data['businesses']=data['businesses'].astype('float64')
data['employees']=data['employees'].astype('float64')


In [ ]:
data.loc[data['businesses'] == 0, 'businesses'] = 1.0
data.loc[data['employees'] == 0, 'employees'] = 1.0



In [ ]:
# And finally choose the Ml model to be applied, among: regression, Classifications, time_series, Clustering, NLP
from pycaret.regression import *

In [ ]:
#droping rows with missing values in labels (because is regression)
data=data.dropna()


In [ ]:
data['income_employee_day']=data['income']*1000000/data['employees']/360
data['employees_business'] = data['employees']/data['businesses']
data['salary_employee_day']=data['Payroll']*1000000/data['employees']/360
data=data.drop(['employees','businesses','Payroll'],axis=1)
data.head()

## Findind the most accurate subdataset
### which would be the best models with whole trainning data? 
Once finding, get the three bes and using plot_model with 'feature', find the most reevant variables to use, and cut the entire dataset.


setup the model

In [ ]:
#You can see below the different considerations about model setup, and correct whatever you need to

selected_models = setup( data = data, 
                        target = "salary_employee_day", session_id=cfg.general_ml.seed,
                        transformation=True,
                        normalize=True,
                        polynomial_features=True,
                        )

In [12]:
best_from_selection= compare_models() 


: 

In [ ]:
tuned_model = tune_model(best_from_selection)


In [ ]:

evaluate_model(tuned_model)
